In [1]:
import os,sys
import torch
import torchvision
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
import torchvision.models.resnet as models 
import torch.nn.functional as F  
from PIL import Image
import h5py
import numpy as np
import scipy.io as sio
import pickle
import pdb
import matplotlib.pyplot as plt
import pandas as pd
import gensim.downloader as api
import torch.optim as optim
import importlib

from DAZLE_Sushree import DAZLE
from AWA2DataLoader import AWA2DataLoader
from helper_func import eval_zs_gzsl,visualize_attention#,get_attribute_attention_stats

ModuleNotFoundError: No module named 'torch'

In [2]:
data_path = 'C:/Sushree/Jio_Institute/Dataset/'
feature_path = 'C:/Sushree/Jio_Institute/Dataset/Animals_with_Attributes2/'

dataloader = AWA2DataLoader(data_path, feature_path, device = None)

C:/Sushree/Jio_Institute/Dataset/
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
AWA2
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
Balance dataloader
_____
C:/Sushree/Jio_Institute/Dataset/Animals_with_Attributes2/feature_map_ResNet_101_AWA2.hdf5
Expert Attr
threshold at zero attribute with negative value


In [3]:
def get_lr(optimizer):
    lr = []
    for param_group in optimizer.param_groups:
        lr.append(param_group['lr'])
    return lr

In [4]:
seed = 200
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)

# define inputs to the network
batch_size = 32
nepoches = 100
niters = dataloader.ntrain * nepoches//batch_size # 73521 for AWA2
dim_f = 2048
dim_v = 300
init_w2v_att = dataloader.w2v_att # load the attribute semantic vectors
att = dataloader.att
att[att<0] = 0
normalize_att = dataloader.normalize_att


trainable_w2v = True
lambda_ = 0.1
#normalize_V=True
#normalize_F=True
#is_conservative=True
bias = 0
prob_prune = 0
uniform_att_1 = False
uniform_att_2 = False


#is_conv=False
#is_bias=True
#non_linear_act=False
#loss_type = 'CE'
#non_linear_emb = False
#is_sigmoid = False

seenclass = dataloader.seenclasses #load seen and unseen data
unseenclass = dataloader.unseenclasses 
desired_mass = 1
report_interval = niters//nepoches

device = None

model = DAZLE(dim_f,dim_v,init_w2v_att,att,normalize_att,
            seenclass,unseenclass,
            lambda_,
            trainable_w2v,normalize_V=True,normalize_F=True,is_conservative=True,
            uniform_att_1=uniform_att_1,uniform_att_2=uniform_att_2,
            prob_prune=prob_prune,desired_mass=desired_mass, is_conv=False,
            is_bias=True)
model.to(device)

setup = {'pmp':{'init_lambda':0.1,'final_lambda':0.1,'phase':0.8},
         'desired_mass':{'init_lambda':-1,'final_lambda':-1,'phase':0.8}}
print(setup)

params_to_update = []
params_names = []
for name,param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        params_names.append(name)
        print("\t",name)
#%%
lr = 0.0001
weight_decay = 0.0001
momentum = 0.
#%%
lr_seperator = 1
lr_factor = 1
print('default lr {} {}x lr {}'.format(params_names[:lr_seperator],lr_factor,params_names[lr_seperator:]))
optimizer  = optim.RMSprop( params_to_update ,lr=lr,weight_decay=weight_decay, momentum=momentum)
print('-'*30)
print('learing rate {}'.format(lr))
print('trainable V {}'.format(trainable_w2v))
print('lambda_ {}'.format(lambda_))
print('optimized seen only')
print('optimizer: RMSProp with momentum = {} and weight_decay = {}'.format(momentum,weight_decay))
print('-'*30)

------------------------------
Configuration
loss_type CE
normalize_V = True: normalize V
normalize_F = True: normalize F
is_conservative = True: training to exclude unseen class [seen upperbound]
Init word2vec
non_linear_act = False: Linear model
loss_att BCEWithLogitsLoss()
Bilinear attention module
******************************
Measure w2v deviation
Compute Pruning loss Parameter containing:
tensor(0)
is_bias = True: Add one smoothing
Second layer attenion conditioned on image features
------------------------------
is_sigmoid = False: No sigmoid on attr score
{'pmp': {'init_lambda': 0.1, 'final_lambda': 0.1, 'phase': 0.8}, 'desired_mass': {'init_lambda': -1, 'final_lambda': -1, 'phase': 0.8}}
	 V
	 W_1
	 W_2
	 W_3
default lr ['V'] 1x lr ['W_1', 'W_2', 'W_3']
------------------------------
learing rate 0.0001
trainable V True
lambda_ 0.1
optimized seen only
optimizer: RMSProp with momentum = 0.0 and weight_decay = 0.0001
------------------------------


C:\Users\Sushree.Behera\Sushree\GZSL_Implementation\DAZLE_Sushree.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.init_w2v_att = F.normalize(torch.tensor(init_w2v_att))
C:\Users\Sushree.Behera\Sushree\GZSL_Implementation\DAZLE_Sushree.py:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.att = nn.Parameter(F.normalize(torch.tensor(att)),requires_grad = False)


In [5]:
best_performance = [0,0,0,0]
for i in range(0,niters):
    model.train()
    optimizer.zero_grad()
    
    cal_epoc = np.ceil((batch_size * i)/dataloader.ntrain)
    
    batch_label, batch_feature, batch_att = dataloader.next_batch(batch_size)
    out_package = model(batch_feature)
    
    in_package = out_package
    in_package['batch_label'] = batch_label
    
    out_package=model.compute_loss(in_package)
    loss, loss_CE, loss_cal = out_package['loss'], out_package['loss_CE'], out_package['loss_cal']
    
    loss.backward()
    optimizer.step()
    if i%report_interval==0:
        print('-'*30)
        # evaluate the model for every report interval
        acc_seen, acc_novel, H, acc_zs = eval_zs_gzsl(dataloader, model, device, bias_seen=-bias, bias_unseen=bias)
        
        if H > best_performance[2]:
            best_performance = [acc_seen, acc_novel, H, acc_zs]
        stats_package = {'epoch': cal_epoc, 'iter':i, 'loss':loss.item(), 'loss_CE':loss_CE.item(),
                         'loss_cal': loss_cal.item(),
                         'acc_seen':best_performance[0], 'acc_novel':best_performance[1], 'H':best_performance[2], 'acc_zs':best_performance[3]}
        
        print(stats_package)

------------------------------
bias_seen 0 bias_unseen 0
{'epoch': 0.0, 'iter': 0, 'loss': 3.941152572631836, 'loss_CE': 3.896059036254883, 'loss_cal': 0.45093637704849243, 'acc_seen': 0, 'acc_novel': 0, 'H': 0, 'acc_zs': 0}
------------------------------
bias_seen 0 bias_unseen 0
{'epoch': 1.0, 'iter': 735, 'loss': 1.1679426431655884, 'loss_CE': 1.0368869304656982, 'loss_cal': 1.3105567693710327, 'acc_seen': 0.6444005370140076, 'acc_novel': 0.5170239806175232, 'H': 0.5737273937327922, 'acc_zs': 0.5957245230674744}
------------------------------
bias_seen 0 bias_unseen 0
{'epoch': 2.0, 'iter': 1470, 'loss': 0.7221055626869202, 'loss_CE': 0.5876328945159912, 'loss_cal': 1.3447265625, 'acc_seen': 0.736900806427002, 'acc_novel': 0.5359684228897095, 'H': 0.6205752389172787, 'acc_zs': 0.6223296523094177}
------------------------------
bias_seen 0 bias_unseen 0
{'epoch': 3.0, 'iter': 2205, 'loss': 0.7071889042854309, 'loss_CE': 0.5941286087036133, 'loss_cal': 1.1306027173995972, 'acc_seen': 

------------------------------
bias_seen 0 bias_unseen 0
{'epoch': 28.0, 'iter': 20580, 'loss': 0.7789356708526611, 'loss_CE': 0.6615577936172485, 'loss_cal': 1.1737788915634155, 'acc_seen': 0.7703193426132202, 'acc_novel': 0.580307126045227, 'H': 0.6619473469862397, 'acc_zs': 0.6695194244384766}
------------------------------
bias_seen 0 bias_unseen 0
{'epoch': 29.0, 'iter': 21315, 'loss': 0.5861824750900269, 'loss_CE': 0.46832162141799927, 'loss_cal': 1.1786084175109863, 'acc_seen': 0.7703193426132202, 'acc_novel': 0.580307126045227, 'H': 0.6619473469862397, 'acc_zs': 0.6695194244384766}
------------------------------
bias_seen 0 bias_unseen 0
{'epoch': 30.0, 'iter': 22050, 'loss': 0.7304123640060425, 'loss_CE': 0.6051203012466431, 'loss_cal': 1.2529202699661255, 'acc_seen': 0.7703193426132202, 'acc_novel': 0.580307126045227, 'H': 0.6619473469862397, 'acc_zs': 0.6695194244384766}
------------------------------
bias_seen 0 bias_unseen 0
{'epoch': 31.0, 'iter': 22785, 'loss': 0.5885374

------------------------------
bias_seen 0 bias_unseen 0
{'epoch': 56.0, 'iter': 41160, 'loss': 0.5539054870605469, 'loss_CE': 0.41676872968673706, 'loss_cal': 1.3713678121566772, 'acc_seen': 0.7703193426132202, 'acc_novel': 0.580307126045227, 'H': 0.6619473469862397, 'acc_zs': 0.6695194244384766}
------------------------------
bias_seen 0 bias_unseen 0
{'epoch': 57.0, 'iter': 41895, 'loss': 0.6565247774124146, 'loss_CE': 0.5319141745567322, 'loss_cal': 1.2461062669754028, 'acc_seen': 0.7703193426132202, 'acc_novel': 0.580307126045227, 'H': 0.6619473469862397, 'acc_zs': 0.6695194244384766}
------------------------------
bias_seen 0 bias_unseen 0
{'epoch': 58.0, 'iter': 42630, 'loss': 0.5437021255493164, 'loss_CE': 0.40989580750465393, 'loss_cal': 1.3380634784698486, 'acc_seen': 0.7703193426132202, 'acc_novel': 0.580307126045227, 'H': 0.6619473469862397, 'acc_zs': 0.6695194244384766}
------------------------------
bias_seen 0 bias_unseen 0
{'epoch': 59.0, 'iter': 43365, 'loss': 0.643273

------------------------------
bias_seen 0 bias_unseen 0
{'epoch': 84.0, 'iter': 61740, 'loss': 0.6211163997650146, 'loss_CE': 0.4808676242828369, 'loss_cal': 1.4024876356124878, 'acc_seen': 0.7703193426132202, 'acc_novel': 0.580307126045227, 'H': 0.6619473469862397, 'acc_zs': 0.6695194244384766}
------------------------------
bias_seen 0 bias_unseen 0
{'epoch': 85.0, 'iter': 62475, 'loss': 0.5780199766159058, 'loss_CE': 0.4276753067970276, 'loss_cal': 1.503446340560913, 'acc_seen': 0.7703193426132202, 'acc_novel': 0.580307126045227, 'H': 0.6619473469862397, 'acc_zs': 0.6695194244384766}
------------------------------
bias_seen 0 bias_unseen 0
{'epoch': 86.0, 'iter': 63210, 'loss': 0.5522638559341431, 'loss_CE': 0.42945435643196106, 'loss_cal': 1.2280950546264648, 'acc_seen': 0.7703193426132202, 'acc_novel': 0.580307126045227, 'H': 0.6619473469862397, 'acc_zs': 0.6695194244384766}
------------------------------
bias_seen 0 bias_unseen 0
{'epoch': 87.0, 'iter': 63945, 'loss': 0.53183412

# Results

========================================================================================================================
Paper
----------
acc_seen: 75.7      

acc_novel: 60.3      

H: 67.1


========================================================================================================================
Original Implementation
-----------------------------------
batch_size = 50

nepoches = 20

lambda_ = 0.1

lr = 0.0001

weight_decay = 0.0001

momentum = 0.

loss: 0.6614841818809509      loss_CE: 0.5430386662483215 (cross-entropy)     loss_cal: 1.184455394744873 (self-calibrate)

acc_seen: 0.7628469467163086     acc_novel: 0.6038219332695007     H: 0.6740823983052283 (harmonic accuracy)

=======================================================================================================================

My Implementation
---------------------------
batch_size = 32

nepoches = 100

lambda_ = 0.1

lr = 0.0001

weight_decay = 0.0001

momentum = 0.

iter: 73500        

loss: 0.606707751750946          loss_CE: 0.4820508360862732 (cross-entropy)    loss_cal: 1.2465689182281494 (self calibration)

acc_seen: 0.7545713782310486               acc_novel: 0.5993922352790833     

H: 0.6680891872758474 (harmonic)           acc_zs: 0.6729567050933838

==========================================================================================================================

My Implementation 2
------------------------------
batch_size = 32

nepoches = 100

lambda_ = 0.1

lr = 0.0001

weight_decay = 0.0001

momentum = 0.

bias_seen 0 bias_unseen 0

epoch: 100.0

iter': 73500

loss: 0.6513152122497559      loss_CE: 0.5178134441375732 (cross-entropy)      loss_cal: 1.3350176811218262 (self calibration)

acc_seen: 0.7703193426132202            acc_novel: 0.580307126045227    

H: 0.6619473469862397 (harmonic)        acc_zs: 0.6695194244384766

==========================================================================================================================
